![](https://lyme-online.co.uk/wp-content/uploads/2019/03/Social-Media-Marketing-Political-Election-Campaign-Propaganda.jpg)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os
import seaborn as sns
#import plotly.express as px
from tqdm import tqdm
import matplotlib.pyplot as plt


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data_folder = "/kaggle/input/local-elections-romania-2020"
data_files = os.listdir(data_folder)

data_df = pd.DataFrame()
for data_file in tqdm(data_files):
    date_hour = data_file.split("_")
    date = date_hour[1]
    hour = date_hour[2].split("-")[0]
    _df = pd.read_csv(os.path.join(data_folder, data_file))
    _df['date'] = date
    _df['hour'] = hour
    data_df = data_df.append(_df)
data_df.head(3)

# Function that creates information about the number of votes by sex and age

In [ ]:
def county_sex_age(data, county, sex):
    
    if sex == "men":
        Men18_24 = data[(data["Judet"]== county) & (data["hour"]=="21")]["Barbati 18-24"].sum()

        Men25_34 = data[(data["Judet"]== county) & (data["hour"]=="21")]["Barbati 25-34"].sum()

        Men35_44 =  data[(data["Judet"]== county) & (data["hour"]=="21")]["Barbati 35-44"].sum()

        Men45_64 =  data[(data["Judet"]== county) & (data["hour"]=="21")]["Barbati 45-64"].sum()

        Men65 =  data[(data["Judet"]== county) & (data["hour"]=="21")]["Barbati 65+"].sum()


        men = pd.DataFrame([Men18_24,Men25_34, Men35_44, Men45_64, Men65 ], index= data.columns[14:19])
        men.columns = ["Value"]
        return(men)
       
    if sex == "women":
        Women18_24 = data[(data["Judet"]== county) & (data["hour"]=="21")]["Femei 18-24"].sum()

        Women25_34 = data[(data["Judet"]== county) & (data["hour"]=="21")]["Femei 25-34"].sum()

        Women35_44 =  data[(data["Judet"]== county) & (data["hour"]=="21")]["Femei 35-44"].sum()

        Women45_64 =  data[(data["Judet"]== county) & (data["hour"]=="21")]["Femei 45-64"].sum()

        Women65 =  data[(data["Judet"]== county) & (data["hour"]=="21")]["Femei 65+"].sum()


        Women = pd.DataFrame([Women18_24, Women25_34, Women35_44, Women45_64, Women65 ], index= data.columns[19:24])
        Women.columns = ["Value"]
        return(Women)

# Example of this function

In [ ]:
County = "BH"
Sex ="men"
county_sex_age(data_df, County , Sex )

# Function that creates pie charts by sex, age and county

In [ ]:
def county_sex_age_pie(county, sex):
    df = county_sex_age(data_df, County , Sex )

    fig1, ax1 = plt.subplots(figsize=(20, 10))
    ax1.pie(df["Value"], labels=df.index, autopct='%1.1f%%',
            shadow=True, startangle=90)
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.legend(fontsize=15)
    plt.rcParams.update({'font.size': 10})
    plt.show()

# Example of this function


In [ ]:
County = "B"
Sex ="men"
county_sex_age_pie(County, Sex)

# Votes by county

In [ ]:
Judet = data_df["Judet"].drop_duplicates().to_list()

votes = []

for i in tqdm(Judet):
    
    votes.append(county_sex_age(data_df,i, "men").sum()[0] + county_sex_age(data_df,i, "women").sum()[0])

In [ ]:
Vote_judet = pd.DataFrame([Judet,votes]).T
Vote_judet.columns = ["Judet", "Votes"]

Vote_judet = Vote_judet.sort_values('Votes', ascending = False)
f, (ax1) = plt.subplots(figsize=(15, 7))
sns.barplot(x=Vote_judet.Judet, y=Vote_judet.Votes, palette="rocket", ax=ax1)
ax1.axhline(0, color="k", clip_on=False)
#ax1.set_ylabel("Sequential") 

# Votes By sex

In [ ]:
votesM = []
votesW = []

for i in tqdm(Judet):
    
    votesM.append(county_sex_age(data_df,i, "men").sum()[0])
    votesW.append(county_sex_age(data_df,i, "women").sum()[0])


# We can notice that there are no significant differences in the number of votes between the global sexes

In [ ]:
fig1, ax1 = plt.subplots(figsize=(18, 8))
ax1.pie([sum(votesM),sum(votesW)], labels=["Men","Women"], autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.legend(fontsize=15)
plt.rcParams.update({'font.size': 16})
plt.show()

# Looking at the distribution by sex and age, we see that the largest proportion of voters are those between 45 and 64 years old.

In [ ]:
sex_age = data_df.columns[14:24]
col = ["Judet"]
col.extend(sex_age)

sex_age_complete = data_df[col].groupby("Judet").agg(sum).sum().sort_values()

fig1, ax1 = plt.subplots(figsize=(18, 8))
ax1.pie(sex_age_complete, labels=sex_age_complete.index, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.legend(fontsize=15)
plt.rcParams.update({'font.size': 16})
plt.show()

# Accumulated votes by county

In [ ]:
votesHour = data_df[["Judet", "LT", "hour"]].groupby(["Judet","hour"]).sum("LT").reset_index()
votesHour.hour = pd.to_numeric(votesHour.hour)
votesHour = votesHour.pivot("hour", "Judet", "LT")

sns.set(rc={'figure.figsize':(15,12)})
g = sns.lineplot(data=votesHour, dashes=False)
g.text(18, 600000, "Judet = B", horizontalalignment='left', size='medium', color='black', weight='semibold')
g.text(18, 45000, "Judet = CV", horizontalalignment='left', size='medium', color='black', weight='semibold')

# We created a function that measures the percentage of votes counted and the total maximum on the permanent and supplementary lists

In [ ]:
def percrnt_part(County, UAT):

    total_Votes = data_df[(data_df["Judet"]== County) & (data_df["UAT"]== UAT) & (data_df["hour"]=="21")]["LT"].sum()

    total_VPLP = data_df[(data_df["Judet"]== County) & (data_df["UAT"]== UAT) & (data_df["hour"]=="21")]["Votanti pe lista permanenta"].sum()
    total_VPLC = data_df[(data_df["Judet"]== County) & (data_df["UAT"]== UAT) & (data_df["hour"]=="21")]["Votanti pe lista complementara"].sum()
    total = total_VPLP + total_VPLC
    return(total_Votes/total)


percrnt_part("AB", "MUNICIPIUL ALBA IULIA")

# We calculate this percentage for the following locations

In [ ]:
Per_part = pd.DataFrame(data_df[["Judet", "UAT"]].drop_duplicates().apply(lambda x: percrnt_part(x['Judet'], x['UAT']), axis=1))
Per_part.columns = ["per"]
Per_part = Per_part.reset_index(drop = True)

In [ ]:
sns.set_style("darkgrid", {"axes.facecolor": ".9"})
tips = sns.load_dataset("tips")
#ax = sns.boxplot(y="per", data=Per_part)
#fig.set_size_inches(, 15)
plt.figure(figsize=(15,12))
ax = sns.swarmplot(y="per", data=Per_part , color = "blue")

#  <span style="color:red">In these places there is a participation percentage greater than 1, which does not make sense, this could be irregularities or outliers.</span>

In [ ]:
X = data_df[["Judet", "UAT"]].drop_duplicates().reset_index(drop= True)
X = pd.concat([X,Per_part], axis = 1)
X[X["per"] > 1]